In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# !pip install dask-ml[complete]
# import dask_ml

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nanoutlierremoveddataset/NaNOutlierRemovedDataset.csv
/kaggle/input/save-the-attack-contest/sample_submission.csv
/kaggle/input/save-the-attack-contest/train_data.csv
/kaggle/input/save-the-attack-contest/test_data.csv


In [2]:
import numpy as np 
import pandas as pd
import dask.dataframe as dd
import gc
usecol=[]


In [3]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            mean = props[col].mean()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mean,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [4]:
columns = list(dd.read_csv("/kaggle/input/nanoutlierremoveddataset/NaNOutlierRemovedDataset.csv").head())
for c in columns:
    if c == "MachineIdentifier":
        usecol.append(c)
    elif c!='PuaMode' and c!='HasDetections' and (c in  columns):
        usecol.append(c)

train_dataset = pd.read_csv("/kaggle/input/nanoutlierremoveddataset/NaNOutlierRemovedDataset.csv",usecols = [i for i in columns if i != 'PuaMode'])
test_dataset = pd.read_csv("/kaggle/input/save-the-attack-contest/test_data.csv",usecols = usecol)
train_dataset,NAlist = reduce_mem_usage(train_dataset) 
test_dataset,NAlist = reduce_mem_usage(test_dataset) 
# print(NAlist)

Memory usage of properties dataframe is : 2748.3414459228516  MB
******************************
Column:  IsBeta
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  RtpStateBitfield
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  IsSxsPassiveMode
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  AVProductStatesIdentifier
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  AVProductsInstalled
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  AVProductsEnabled
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  HasTpm
dtype before:  int64
dtype after:  uint8
******************************
***********************

In [5]:
train_dataset.drop(columns=['AvSigVersion','Census_OSVersion','OsBuildLab'],inplace=True)
test_dataset.drop(columns=['AvSigVersion','Census_OSVersion','OsBuildLab'],inplace=True)

train_dataset.drop(columns=['MachineIdentifier'],inplace = True)
y_train = train_dataset['HasDetections']
train_dataset.drop(columns= 'HasDetections',inplace = True)
categorical_columns,numerical_columns = [],[]
for c in train_dataset.columns:
    if train_dataset[c].dtype == 'object':
        categorical_columns.append(c)
    else:
        numerical_columns.append(c)

In [6]:
mean,mode = {},{}
for col in train_dataset.columns:
    if train_dataset[col].dtype=='object':
#         train_dataset[col] = train_dataset[col].astype('category')
        mode[col]=train_dataset[col].mode()
#     elif train_dataset[col].dtype.name == 'category':
#         mode[col]= train_dataset[col].mode()
    else :
        mean[col] = train_dataset[col].mean()

for c in test_dataset.columns:
    if test_dataset[c].isnull().any():
        print(c)
        if test_dataset[c].dtype == 'object':
            test_dataset[c].fillna(mode[c],inplace = True)
        else:
            test_dataset[c].fillna(mean[c],inplace = True)

Census_PrimaryDiskTypeName
Census_ChassisTypeName
Census_PowerPlatformRoleName


In [7]:
for c in train_dataset.columns:
    if train_dataset[c].dtype == 'object':
        print(c,train_dataset[c].nunique())
for c in train_dataset.columns:
    print(c)

ProductName 6
EngineVersion 69
AppVersion 100
Platform 4
Processor 3
OsVer 49
OsPlatformSubRelease 9
SkuEdition 8
Census_MDC2FormFactor 12
Census_DeviceFamily 2
Census_PrimaryDiskTypeName 4
Census_ChassisTypeName 48
Census_PowerPlatformRoleName 9
Census_OSArchitecture 3
Census_OSBranch 20
Census_OSEdition 28
Census_OSSkuName 26
Census_OSInstallTypeName 9
Census_OSWUAutoUpdateOptionsName 6
Census_GenuineStateName 5
Census_ActivationChannel 6
Census_FlightRing 10
ProductName
EngineVersion
AppVersion
IsBeta
RtpStateBitfield
IsSxsPassiveMode
AVProductStatesIdentifier
AVProductsInstalled
AVProductsEnabled
HasTpm
CountryIdentifier
CityIdentifier
GeoNameIdentifier
LocaleEnglishNameIdentifier
Platform
Processor
OsVer
OsBuild
OsSuite
OsPlatformSubRelease
SkuEdition
IsProtected
AutoSampleOptIn
SMode
IeVerIdentifier
Firewall
UacLuaenable
Census_MDC2FormFactor
Census_DeviceFamily
Census_OEMNameIdentifier
Census_OEMModelIdentifier
Census_ProcessorCoreCount
Census_ProcessorManufacturerIdentifier
Cen

In [8]:
y_test_id = test_dataset['MachineIdentifier']
test_dataset.drop(columns = ['MachineIdentifier'],inplace = True)

In [9]:
train_dataset.shape

(4867981, 69)

In [10]:
test_dataset.shape

(1783540, 69)

In [11]:
# entries = []
# for c in categorical_columns:
#     if(c == 'MachineIdentifier'):
#         continue
#     print(c)
#     t1 = list(test_dataset[c].unique())
#     t2 = list(train_dataset[c].unique())
# #     print(t1)
# #     print('\n')
# #     print(t2)
# #     dic = []

#     for v in t1:
#         if v not in t2:
#             print(v," not in train for column ",c)

#             new_entry = []
#             for col in train_dataset.columns:
#                 if col == c:
#                     new_entry.append(str(v))
#                 elif train_dataset[col].dtype == 'object':
#                     new_entry.append(str(mode[col]))
#                 else :
#                     new_entry.append(mean[col])
#             entries.append(new_entry)
            
# df = pd.DataFrame(entries,columns = list(train_dataset.columns))
# train_dataset = train_dataset.append(df,ignore_index=True)
# #             print(new_entry)


In [12]:
train_dataset = pd.get_dummies(train_dataset)
test_dataset = pd.get_dummies(test_dataset)
train_dataset,NAlist = reduce_mem_usage(train_dataset) 
test_dataset,NAlist = reduce_mem_usage(test_dataset) 
train_dataset.to_csv("finalTrainDataset.csv",index=False)
test_dataset.to_csv("finalTestDataset.csv",index=False)

Memory usage of properties dataframe is : 2697.274292945862  MB
******************************
Column:  IsBeta
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  RtpStateBitfield
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  IsSxsPassiveMode
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  AVProductStatesIdentifier
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  AVProductsInstalled
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  AVProductsEnabled
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  HasTpm
dtype before:  uint8
dtype after:  uint8
******************************
************************

In [13]:
#normalize 

# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# train_dataset = scaler.fit_transform(train_dataset)
# gc.collect()


In [14]:
# test_dataset = test_dataset.transform(test_dataset)
# gc.collect()

In [15]:
# train_dataset = pd.DataFrame(train_dataset)
# test_dataset = pd.DataFrame(test_dataset)

# train_dataset,NAlist = reduce_mem_usage(train_dataset)
# test_dataset,NAlist = reduce_mem_usage(test_dataset)
# gc.collect()

In [16]:
# from sklearn.decomposition import IncrementalPCA
# pca = IncrementalPCA(n_components=7,copy=False,)
# train_dataset = pca.fit(train_dataset)
# train_dataset = pca.transform(train_dataset)
# reduce_mem_usage(train_dataset)
# test_dataset = pca.transform(test_dataset)
# reduce_mem_usage(test_dataset)
# gc.collect()

In [17]:
#scree plots 
#xgboost
#adaboost
#gradientboost

In [18]:
#scree plots

# PC_values = np.arange(pca.n_components_) + 1
# plt.plot(PC_values, pca.explained_variance_ratio_, 'ro-', linewidth=2)
# plt.title('Scree Plot')
# plt.xlabel('Principal Component')
# plt.ylabel('Proportion of Variance Explained')
# plt.show()

In [19]:
y_train.to_csv("yTrain.csv",index=False)
y_test_id.to_csv("TestID.csv",index=False)